In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pretty_midi
import librosa

In [ ]:
f0_path = '../../data/2019_DagstuhlRecording/Dagstuhl Dataset/Annotations/DLI_QuartetB_Take4_PYIN/'
sop_path = f0_path + 'DLI_QuartetB_Take4_S1_LRX_vamp_pyin_pyin_smoothedpitchtrack.csv'
alt_path = f0_path + 'DLI_QuartetB_Take4_A2_LRX_vamp_pyin_pyin_smoothedpitchtrack.csv'
ten_path = f0_path + 'DLI_QuartetB_Take4_T2_LRX_vamp_pyin_pyin_smoothedpitchtrack.csv'
bas_path = f0_path + 'DLI_QuartetB_Take4_B2_LRX_vamp_pyin_pyin_smoothedpitchtrack.csv'

midi_path = '../../data/2019_DagstuhlRecording/Dagstuhl Dataset/Annotations/SheetMusic_aligned_AudioLabsBeats/Bruckner_WAB023_BrianMarble-CPDL_warpedTo_DLI_QuartetB_Take4_Stereo_STM.mid'

In [ ]:
# load files
sop = pd.read_csv(sop_path, header=None).values
alt = pd.read_csv(alt_path, header=None).values
ten = pd.read_csv(ten_path, header=None).values
bas = pd.read_csv(bas_path, header=None).values

# get onset, offset, MIDI center frequency from aligned MIDI file
patt = pretty_midi.PrettyMIDI(midi_path)
sop_MIDI = np.empty((0, 3))
alt_MIDI = np.empty((0, 3))
ten_MIDI = np.empty((0, 3))
bas_MIDI = np.empty((0, 3))

for note in patt.instruments[0].notes:
    sop_MIDI = np.vstack((sop_MIDI, (note.start, note.end, 440*2**((note.pitch-69)/12))))
for note in patt.instruments[1].notes:
    alt_MIDI = np.vstack((alt_MIDI, (note.start, note.end, 440*2**((note.pitch-69)/12))))
for note in patt.instruments[2].notes:
    ten_MIDI = np.vstack((ten_MIDI, (note.start, note.end, 440*2**((note.pitch-69)/12))))
for note in patt.instruments[3].notes:
    bas_MIDI = np.vstack((bas_MIDI, (note.start, note.end, 440*2**((note.pitch-69)/12))))

In [ ]:
# visualize trajectories
#%matplotlib notebook
plt.figure()
plt.plot(sop[:, 0], sop[:, 1], marker='.', linestyle='')
plt.plot(alt[:, 0], alt[:, 1], marker='.', linestyle='')
plt.plot(ten[:, 0], ten[:, 1], marker='.', linestyle='')
plt.plot(bas[:, 0], bas[:, 1], marker='.', linestyle='')
plt.ylim((0, 800))


In [ ]:
def compute_notewise_f0_mean(contour, MIDI_info):
    notewise_mean = np.zeros(contour.shape)
    notewise_mean[:, 0] = contour[:, 0]
    
    contour_MIDI = np.zeros(contour.shape)
    contour_MIDI[:, 0] = contour[:, 0]
    
    for i in range(MIDI_info.shape[0]):
        # get indices of trajectory
        t_start_idx = np.argmin(np.abs(contour[:, 0] - MIDI_info[i, 0]))
        t_end_idx = np.argmin(np.abs(contour[:, 0] - MIDI_info[i, 1]))
        
        note_len_idx = t_end_idx - t_start_idx
        notewise_mean[t_start_idx:t_end_idx, 1] = [np.mean(contour[t_start_idx:t_end_idx, 1])]*note_len_idx
        contour_MIDI[t_start_idx:t_end_idx, 1] = [MIDI_info[i, 2]]*note_len_idx
    
    notewise_mean[notewise_mean[:, 1] == 0, 1] = np.nan
    contour_MIDI[contour_MIDI[:, 1] == 0, 1] = np.nan
    
    return notewise_mean, contour_MIDI

In [ ]:
# get mean of trajectories in note regions
sop_mean, sop_MIDI_c = compute_notewise_f0_mean(np.abs(sop), sop_MIDI)
alt_mean, alt_MIDI_c = compute_notewise_f0_mean(np.abs(alt), alt_MIDI)
ten_mean, ten_MIDI_c = compute_notewise_f0_mean(np.abs(ten), ten_MIDI)
bas_mean, bas_MIDI_c = compute_notewise_f0_mean(np.abs(bas), bas_MIDI)

In [ ]:
plt.figure()
plt.plot(sop_MIDI_c[:, 0], sop_MIDI_c[:, 1], color='k', marker='.', linestyle='')
plt.plot(sop_MIDI_c[:, 0], alt_MIDI_c[:, 1], color='k', marker='.', linestyle='')
plt.plot(sop_MIDI_c[:, 0], ten_MIDI_c[:, 1], color='k', marker='.', linestyle='')
plt.plot(sop_MIDI_c[:, 0], bas_MIDI_c[:, 1], color='k', marker='.', linestyle='')

plt.plot(sop[:, 0], np.abs(sop[:, 1]), marker='.', linestyle='')
plt.plot(alt[:, 0], np.abs(alt[:, 1]), marker='.', linestyle='')
plt.plot(ten[:, 0], np.abs(ten[:, 1]), marker='.', linestyle='')
plt.plot(bas[:, 0], np.abs(bas[:, 1]), marker='.', linestyle='')
plt.ylim((0, 800))

In [ ]:
# compute deviation between score and mean f0
sop_dev = 1200*np.log2(sop_mean[:, 1] / sop_MIDI_c[:, 1])
alt_dev = 1200*np.log2(alt_mean[:, 1] / alt_MIDI_c[:, 1]) 
ten_dev = 1200*np.log2(ten_mean[:, 1] / ten_MIDI_c[:, 1]) 
bas_dev = 1200*np.log2(bas_mean[:, 1] / bas_MIDI_c[:, 1])

In [ ]:
# visualize mean
sop_act = np.isnan(sop_MIDI_c[:, 1])*1
alt_act = np.isnan(alt_MIDI_c[:, 1])*1
ten_act = np.isnan(ten_MIDI_c[:, 1])*1
bas_act = np.isnan(bas_MIDI_c[:, 1])*1
mean_dev = (sop_dev + alt_dev + ten_dev + bas_dev)/ (sop_act + alt_act + ten_act + bas_act)
print(mean_dev)

In [ ]:
# visualize individual voices
#%matplotlib notebook

fig = plt.figure(figsize=(12, 7))
plt.axhline(y=0, color='k')
#plt.plot(sop_mean[:, 0], mean_dev, color='k', marker='.', linestyle='')
plt.plot(sop_mean[:, 0], sop_dev, marker='.', linestyle='')
plt.plot(alt_mean[:, 0], alt_dev, marker='.', linestyle='')
plt.plot(ten_mean[:, 0], ten_dev, marker='.', linestyle='')
plt.plot(bas_mean[:, 0], bas_dev, marker='.', linestyle='')
plt.ylim((-500, 200))
plt.legend(('', 'Soprano', 'Alto', 'Tenor', 'Bass'))

In [ ]:
import pywebaudioplayer as pwa
import IPython.display
import os

# temporary paths
temp_audio_path = './player.wav'
temp_figure_path = './player.png'

# delete existing files
if os.path.isfile(temp_audio_path):
    os.remove(temp_audio_path)
if os.path.isfile(temp_figure_path):
    os.remove(temp_figure_path)

x, fs = librosa.load('../../data/2019_DagstuhlRecording/Dagstuhl Dataset/Audio/norm-downsampled/DLI_QuartetB_Take4_Stereo_STM.wav')
ts2 = pwa.trackswitch([{'title': 'Audio', 'samples': (x, fs), 'path': temp_audio_path}], seekable_image=(fig, './player.png'), repeat=True)
IPython.display.HTML(ts2)

In [ ]:
from IPython.display import HTML

HTML(filename="./player.html")